## Энергетический оракул
Ноутбук команды #12

Работа выполнена на основе модели LightGBM


### 1. Подготовка данных

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random

import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

import re

from tqdm import tqdm
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True )

from xgboost.callback import TrainingCallback
import time


random_state = 12345
NUM_ITERATIONS = 5000

FEATURES = 'base feature'

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

#### 1.1 Функции для расшифровки прогноза погоды в колонке 'weather_pred'

In [3]:
# Расшифровка прогноза в колонке 'weather_pred'

# функция формирует колонки 'cloudy', 'rainy', 'windy', 'clear', 'rain_probability', 'has_rain_probability'
# в колонках число, которое 0 при отсутсвии упоминания явления в weather_pred или степень упоминания
# функция дает в колонках номер первого списка, элемент которого есть в строке плюс 1
# списки cloudy_list, rainy_list, windy_list, clear_list можно модифицировать
# соответственно, можно экспериментировать с расположением значений в списках
# например, сейчас 'дождь', 'снег', 'д+сн' - первая степень  дождя, а 'гроз', 'ливень' - вторая
# а можно сделать снег второй, а грозу с ливнем убрать в третью
# также сделал отдельный список для "ясности", чтобы выделить 'ясно' и 'солнечно'

def in_what_list(weather, big_list):
    for list_number, small_list in enumerate(big_list):
        if any(word in weather for word in small_list):
            return list_number+1
    return 0

def weather_split2(row):
    weather = row['weather_pred']
    cloudy_list = [['проясн', 'пер.об.', 'п/об'], ['пасм', 'обл']]
    rainy_list = [['дождь', 'снег', 'д+сн'], ['гроз', 'ливень']]
    windy_list = [['вет'],['штор']]
    clear_list = [['проясн'], ['ясно'], ['солнеч']]
    numbers = re.findall(r'\d+', weather)
    cloudy = in_what_list(weather, cloudy_list)
    rainy = in_what_list(weather, rainy_list)
    windy = in_what_list(weather, windy_list)
    clear = in_what_list(weather, clear_list)
    rain_probability = 0 if len(numbers)==0 else int(numbers[0])
    has_rain_probability = int(len(numbers)==0)
    return cloudy, rainy, windy, clear, rain_probability, has_rain_probability

def fill_weather_columns(df):
    df['weather_pred'] = df['weather_pred'].fillna('')
    df['cloudy'], df['rainy'], df['windy'], df['clear'], df['rain_probability'], df['has_rain_probability'] = \
                zip(*df.apply(weather_split2, axis=1))
    return df

#### 1.2 Функции для загрузки данных о ВВП 
данные загружаются из файла 'data/VVP.csv'

Некоторые научные работы указывают на прямую связь величины потребления электричества и показателя ВВП, который отражает ситуацию в экономике. Данные по экономике публикуются различными министерствами с разной периодичностью. Для использования в работе были взяты фактические данные по ВВП с сайта investing, который агрегирует публикации Минэкономразвития. Данные за месяц побликуются с месячной задержкой, поэтому модель использует для прогнозирования данные за прошлые месяцы, которые известны.   
  
Ссылка на данные: https://ru.investing.com/economic-calendar/russian-monthly-gdp-407


In [4]:
# Функция добавляет данные о ВВП из файла 'data/VVP.csv' в датасет

def add_vvp2(data, file_source = 'data/VVP.csv'):
    """
    сырой датафрем подаем на вход
    """
    # обработаем файл с динамикой ВВП
    vvp = pd.read_csv(file_source)
    # преобразуем дату файла-источника в формат datetime64 и дропнем один столбик
    vvp['date'] = pd.to_datetime(vvp['date'], format ='%Y-%m-%d %H:%M:%S')
    vvp.drop('for_month',axis=1,inplace=True) 
    
    # обработаем основной фрейм - создадим столбец для соединения, который потом удалим
    data['date_temp'] = pd.to_datetime(data['date'], format = '%Y-%m-%d' )
    data['date_temp'] = data['date_temp'] + pd.to_timedelta(data['time'] , 'H')
    
    # соединяем основной фрейм и ВВП по дате объявления показтеля ВВП
    for idx in reversed(vvp.index):
        data.loc[data['date_temp']>=vvp.date[idx],'VVP'] = vvp.VVP_perc[idx]
        
    data.drop('date_temp',axis=1,inplace=True)   

    return data

#### 1.3 Функции для загрузки архива данных о фактической погоде
данные загружаются из файла 'data/preprocessing_loaded_table.csv'

Изначально данные для формирования таблицы "preprocessing_loaded_table" были взяты из с сайта [https://rp5.ru](https://rp5.ru/Архив_погоды_в_Храброво,_им._императрицы_Елизаветы_Петровны_(аэропорт),_METAR), где хранятся архивы погоды в аэрапорту Калининграда, за период с 31.12.2018 по 30.09.2023

Описание данных в таблице:
- Местное время в Храброво / им. императрицы Елизаветы Петровны (аэропорт) - Дата / Местное время
- T -  Темпиратура воздуха
- Po - Давление на уровне станции
- P - Давление приведённое к уровню моря
- U - Относительная влажность
- DD - Направление ветра
- Ff - Скорость ветра
- ff10 - Максимальное значение порыва ветра
- WW - Особое явление текущей погоды (осадки)
- W'W' - Явление недавней погоды, имеющее оперативное значение
- с - Общая облачность
- VV - Горизонтальная дальность видимости
- Td - Темпиратура точки росы

Данные, которые были взяты из данной таблицы и загружаются из 'data/preprocessing_loaded_table.csv':
- P - не подверглось изменению
- U - не подверглось изменению
- Td - не подверглась изменению

 WW - разделили на 4 категории:
- Нет осадков (где были пропуски)
- слабый дождь
- сильный дождь
- снег

DD - создали 4 столбца, соответствующих сторонам горизонта, которые принимали значения 0; 0.5 и 1 в зависимости от силы ветра в конкретном направлении
- N - north
- S - south
- W - west
- E - east

В дальнейшем эти данные использовались с лагом в сутки: в поля на завтрашний день записывались данные сегодняшнего.

In [5]:
# Функции для работы с данными о фактической погоде из 'data/preprocessing_loaded_table.csv'

# Кодировка информации об осадках из колонки WW
def true_weather_WW_replace(ww):
    if ww=='нет осадков':
        return 0
    elif ww=='слабый дождь':
        return 1
    elif (ww=='сильный дождь') or (ww=='снег'):
        return 2
    else:
        return 3

# Вычисление Timestamp из даты и времени
def row_plus_hours_to_index(row):
    return row['date'] + pd.to_timedelta(row['time'] , 'H')

# Функция для сдвига на сутки (в скачанном датасете разбивка по 30 мин, поэтому timeshift=48)
def shift_features_fact(df, timeshift=48):
    list_fact_columns=list(df.columns)
    list_fact_columns.remove('date_tw')
    new_df = df.copy()
    for column in list_fact_columns:
        new_df[column] = new_df[column].shift(timeshift)

    return new_df

In [6]:
# Функция для вычисления метрики mae по дням из почасовых массивов данных

def mae_day(y_true, y_pred):
    y_true_copy = pd.DataFrame(y_true).reset_index(drop=True)
    y_true_copy['day'] = y_true_copy.index // 24
    y_true_grouped = y_true_copy.groupby(by='day').sum()   
    y_pred_copy = pd.DataFrame(y_pred).reset_index(drop=True)
    y_pred_copy['day'] = y_pred_copy.index // 24
    y_pred_grouped = y_pred_copy.groupby(by='day').sum()
    
    return mean_absolute_error(y_true_grouped, y_pred_grouped)
# Функция для вычисления метрик по дням из почасовых массивов данных

def metrics_hour(y_true, y_pred):

    
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, mape, r2

#### 1.5 Чтение файлов с данными
Данные объединяются в один датасет

In [7]:
# читаем исходные датасеты и складываем в один
train_ds = pd.read_csv('data/train_dataset.csv')
test_ds = pd.read_csv('data/test_dataset.csv')
train_ds = pd.concat([train_ds, test_ds])

# запоминаем дату начала тестовых данных, потом также поступим и с закрытым датасетом
open_test_begin = pd.to_datetime(test_ds['date']).min()
open_test_end = pd.to_datetime(test_ds['date']).max() + pd.to_timedelta(1,'d')
print('начало открытого теста:', open_test_begin, '    конец открытого теста:', open_test_end)

начало открытого теста: 2023-04-01 00:00:00     конец открытого теста: 2023-08-01 00:00:00


#### 1.6 Формирование колонок с производными от даты

In [8]:
# преобразуем дату и делаем из нее колонки
train_ds['date'] = pd.to_datetime(train_ds['date'])
train_ds['year'] = train_ds['date'].dt.year
train_ds['month'] = train_ds['date'].dt.month
train_ds['day_of_week'] = train_ds['date'].dt.dayofweek
train_ds['day'] = train_ds['date'].dt.day
train_ds['day_of_year'] = train_ds['date'].dt.dayofyear

In [9]:
train_ds.sample(5)

,date,time,target,temp,temp_pred,weather_pred,weather_fact,year,month,day_of_week,day,day_of_year
381,2023-04-16,21,524.178,5.7,6.0,ясно,"пасм, слаб.дождь",2023,4,6,16,106
34818,2022-12-21,18,694.846,2.0,3.0,"пасм, ветер, 74% дождь",дымка,2022,12,2,21,355
33157,2022-10-13,13,519.836,14.6,14.0,облачно,облачно,2022,10,3,13,286
34541,2022-12-10,5,503.541,-4.9,-4.0,пасмурно,"пасм, дымка",2022,12,5,10,344
25896,2021-12-15,0,567.499,3.0,3.0,"пасм, 16%","пасм, морось",2021,12,2,15,349


#### 1.7 Подгрузка Auggumentaci в праздниках


df_holidays = pd.read_csv('data/holidays_aug.csv')
df_holidays['date'] = pd.to_datetime(df_holidays['date'])
# Добавление данных о праздниках из файла 'data/holidays.csv'


print('размер DS', train_ds.shape, 'дубликатов - ', train_ds.duplicated().sum())
# Assuming df_holidays and train_ds are your dataframes
train_ds = pd.merge(train_ds, df_holidays, on='date', how='left')
print('размер DS', train_ds.shape, 'дубликатов - ', train_ds.duplicated().sum())
# Fill NaN values with 0
train_ds['holidays'].fillna(0, inplace=True)
train_ds['preholidays'].fillna(0, inplace=True)

# Convert to int
train_ds['holidays'] = train_ds['holidays'].astype(int)
train_ds['preholidays'] = train_ds['preholidays'].astype(int)
print('размер DS', train_ds.shape, 'дубликатов - ', train_ds.duplicated().sum())

#### 1.7.1 Подгрузка данных о праздниках на весь DS праздниках

In [10]:

df_holidays_true = pd.read_csv('data/holidays_true.csv')
df_holidays_true['date'] = pd.to_datetime(df_holidays_true['date'])
# Добавление данных о праздниках из файла 'data/holidays.csv'
print('размер DS', train_ds.shape, 'дубликатов - ', train_ds.duplicated().sum())
# Assuming df_holidays and train_ds are your dataframes
train_ds = pd.merge(train_ds, df_holidays_true, on='date', how='left')

# Fill NaN values with 0
train_ds['holidays_true'].fillna(0, inplace=True)
train_ds['preholidays_true'].fillna(0, inplace=True)

# Convert to int
train_ds['holidays_true'] = train_ds['holidays_true'].astype(int)
train_ds['preholidays_true'] = train_ds['preholidays_true'].astype(int)
print('размер DS', train_ds.shape, 'дубликатов - ', train_ds.duplicated().sum())

размер DS (40152, 12) дубликатов -  0
размер DS (40152, 14) дубликатов -  0


In [11]:
train_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40152 entries, 0 to 40151
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              40152 non-null  datetime64[ns]
 1   time              40152 non-null  int64         
 2   target            40152 non-null  float64       
 3   temp              40152 non-null  float64       
 4   temp_pred         40040 non-null  float64       
 5   weather_pred      40040 non-null  object        
 6   weather_fact      40151 non-null  object        
 7   year              40152 non-null  int32         
 8   month             40152 non-null  int32         
 9   day_of_week       40152 non-null  int32         
 10  day               40152 non-null  int32         
 11  day_of_year       40152 non-null  int32         
 12  holidays_true     40152 non-null  int64         
 13  preholidays_true  40152 non-null  int64         
dtypes: datetime64[ns](1), 

#### 1.8 Формирование колонок со значением целевого признака в предыдущие дни

In [12]:
# Добавление колонок с временными лагами

# создаем столбец 'temp_last_day'
train_ds['temp_last_day'] = train_ds['temp'].shift(24)
print('размер DS', train_ds.shape, 'дубликатов - ', train_ds.duplicated().sum())
# заполняем пропущенные значения в 'temp_last_day'
train_ds['temp_last_day'].fillna(method='bfill', inplace=True)

# создаем столбцы с временными лагами для 'target'
lags = [24, 48, 72, 7*24, 14*24]
for lag in lags:
    train_ds[f'target_lag_{lag}'] = train_ds['target'].shift(lag)

# заполняем пропущенные значения в столбцах с лагами
for lag in lags:
    train_ds[f'target_lag_{lag}'].fillna(0, inplace=True)

print('размер DS', train_ds.shape, 'дубликатов - ', train_ds.duplicated().sum())

размер DS (40152, 15) дубликатов -  0
размер DS (40152, 20) дубликатов -  0


/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_37729/2320664940.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_ds['temp_last_day'].fillna(method='bfill', inplace=True)


#### 1.9 Формирование колонок с ВВП и данными о погоде посредством ранее описанных функций

In [13]:
# применяем функцию добавления ВВП
train_ds = add_vvp2(train_ds)

# Расшифровка прогноза в колонке 'weather_pred'
train_ds = fill_weather_columns(train_ds)


# Читаем файл с архивом фактической погоды
df_true_weather = pd.read_csv('data/preprocessing_loaded_table.csv')
display(df_true_weather)

# Форматируем колонки
df_true_weather['WW'] = df_true_weather['WW'].apply(true_weather_WW_replace)
df_true_weather['date'] = pd.to_datetime(df_true_weather['date'])
df_true_weather = df_true_weather.rename(columns={'date':'date_tw'})
# Применяем сдвиг на сутки, чтобы не заглядывать в будущее
df_true_weather = shift_features_fact(df_true_weather)
# Добавляем в датасет
train_ds['date_hours'] = train_ds.apply(row_plus_hours_to_index, axis=1)
train_ds = train_ds.merge(df_true_weather, left_on='date_hours', right_on='date_tw')
train_ds = train_ds.drop(['date_hours', 'date_tw'], axis=1)

,date,P,U,WW,Td,N,S,W,E
0,2018-12-31 00:00:00,763.5,100.0,слабый дождь,2.0,1.0,0.0,0.0,0.0
1,2018-12-31 00:30:00,764.3,93.0,слабый дождь,1.0,1.0,0.0,0.0,0.5
2,2018-12-31 01:00:00,764.3,93.0,слабый дождь,1.0,1.0,0.0,0.0,0.0
3,2018-12-31 01:30:00,765.0,93.0,слабый дождь,2.0,1.0,0.0,0.0,0.0
4,2018-12-31 02:00:00,765.0,93.0,нет осадков,2.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
82146,2023-09-30 21:30:00,763.5,82.0,нет осадков,12.0,0.0,0.0,1.0,0.0
82147,2023-09-30 22:00:00,763.5,82.0,нет осадков,12.0,0.5,0.0,1.0,0.0
82148,2023-09-30 22:30:00,763.5,77.0,сильный дождь,11.0,0.0,0.0,1.0,0.0
82149,2023-09-30 23:00:00,763.5,94.0,сильный дождь,13.0,0.5,0.0,1.0,0.0


#### 1.10 Демонстрация сформированного датасета

In [14]:
# Итоговый набор колонок
train_ds.columns

Index(['date', 'time', 'target', 'temp', 'temp_pred', 'weather_pred',
       'weather_fact', 'year', 'month', 'day_of_week', 'day', 'day_of_year',
       'holidays_true', 'preholidays_true', 'temp_last_day', 'target_lag_24',
       'target_lag_48', 'target_lag_72', 'target_lag_168', 'target_lag_336',
       'VVP', 'cloudy', 'rainy', 'windy', 'clear', 'rain_probability',
       'has_rain_probability', 'P', 'U', 'WW', 'Td', 'N', 'S', 'W', 'E'],
      dtype='object')

In [15]:
train_ds.head()

,date,time,target,temp,temp_pred,weather_pred,weather_fact,year,month,day_of_week,...,rain_probability,has_rain_probability,P,U,WW,Td,N,S,W,E
0,2019-01-01,0,481.510,2.9,2.0,"пасм, ветер",ветер,2019,1,1,...,0,1,763.5,100.0,1.0,2.0,1.0,0.0,0.0,0.0
1,2019-01-01,1,462.872,2.9,2.0,"пасм, ветер",ветер,2019,1,1,...,0,1,764.3,93.0,1.0,1.0,1.0,0.0,0.0,0.0
2,2019-01-01,2,449.718,2.9,2.0,"пасм, ветер",ветер,2019,1,1,...,0,1,765.0,93.0,0.0,2.0,1.0,0.0,0.0,0.0
3,2019-01-01,3,430.908,4.3,2.0,"пасм, ветер","ветер, пасм",2019,1,1,...,0,1,765.8,87.0,0.0,1.0,1.0,0.0,0.0,0.0
4,2019-01-01,4,415.163,4.3,2.0,"пасм, ветер","ветер, пасм",2019,1,1,...,0,1,766.6,87.0,0.0,1.0,1.0,0.0,0.0,0.0


#### 1.11.1 Добавление среднего за час предыдущего дня

In [16]:

def mean_evening(values, evening=19):
    return values[evening:].mean()

evening_slices = [0, 19, 22]
    
for evening_slice in evening_slices:
    train_ds[['last_evening_avg_target_'+str(evening_slice), 'last_evening_avg_temp_'+str(evening_slice)]] = \
        train_ds[['date', 'target', 'temp']].groupby(by='date').transform(mean_evening, evening=evening_slice).shift(24)

In [17]:
train_ds.columns

Index(['date', 'time', 'target', 'temp', 'temp_pred', 'weather_pred',
       'weather_fact', 'year', 'month', 'day_of_week', 'day', 'day_of_year',
       'holidays_true', 'preholidays_true', 'temp_last_day', 'target_lag_24',
       'target_lag_48', 'target_lag_72', 'target_lag_168', 'target_lag_336',
       'VVP', 'cloudy', 'rainy', 'windy', 'clear', 'rain_probability',
       'has_rain_probability', 'P', 'U', 'WW', 'Td', 'N', 'S', 'W', 'E',
       'last_evening_avg_target_0', 'last_evening_avg_temp_0',
       'last_evening_avg_target_19', 'last_evening_avg_temp_19',
       'last_evening_avg_target_22', 'last_evening_avg_temp_22'],
      dtype='object')

#### 1.11 Исключение лишних колонок

In [18]:
# Отбираем признаки. Все лишние колонки здесь отбрасываем, кроме 'date', которую уберем позже 

feature_cols = list(train_ds.columns)

# выбрасываем взгляд в прошлое и расшифрованную погоду
drop_list = ['target', 'day_of_year', 'weather_pred', 'weather_fact', 'temp']

# выбрасываем признаки, найденные процедурно в процессе оптимизации
# КОМАНДЕ: здесь можно добавлять признаки на выброс с целью оптимизации
drop_list = drop_list + ['target_lag_48', 'target_lag_168'] #, 'temp_pred'] #, 'target_lag_336'] 

for name in drop_list:
    feature_cols.remove(name)

# Итоговый список признаков
feature_cols

['date',
 'time',
 'temp_pred',
 'year',
 'month',
 'day_of_week',
 'day',
 'holidays_true',
 'preholidays_true',
 'temp_last_day',
 'target_lag_24',
 'target_lag_72',
 'target_lag_336',
 'VVP',
 'cloudy',
 'rainy',
 'windy',
 'clear',
 'rain_probability',
 'has_rain_probability',
 'P',
 'U',
 'WW',
 'Td',
 'N',
 'S',
 'W',
 'E',
 'last_evening_avg_target_0',
 'last_evening_avg_temp_0',
 'last_evening_avg_target_19',
 'last_evening_avg_temp_19',
 'last_evening_avg_target_22',
 'last_evening_avg_temp_22']

#### 1.11.2 Добавление среднего за час предыдущего дня

In [19]:
FEATURE_WINDOW_SIZE = 24
feature_cols_no_date = feature_cols.copy()
feature_cols_no_date.remove('date')


for lag in range(1,FEATURE_WINDOW_SIZE):
    for column in feature_cols_no_date:
        train_ds[column+'_'+str(lag)] = train_ds[column].shift(lag)

/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_37729/4047160460.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_ds[column+'_'+str(lag)] = train_ds[column].shift(lag)
/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_37729/4047160460.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_ds[column+'_'+str(lag)] = train_ds[column].shift(lag)
/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_37729/4047160460.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the res

#### 1.11.2 Добавление лагов за час 

In [20]:
target_lags=[1, 5, 9]

for lag in target_lags:
    train_ds['target_'+str(lag)] = train_ds['target'].shift(lag).where(train_ds['time']<lag, np.NaN)
    train_ds['temp_'+str(lag)] = train_ds['temp'].shift(lag).where(train_ds['time']<lag, np.NaN)

/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_37729/64960772.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_ds['target_'+str(lag)] = train_ds['target'].shift(lag).where(train_ds['time']<lag, np.NaN)
/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_37729/64960772.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_ds['temp_'+str(lag)] = train_ds['temp'].shift(lag).where(train_ds['time']<lag, np.NaN)
/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_37729/64960772.py:4: PerformanceW

In [21]:
feature_cols = list(train_ds.columns)
for name in drop_list:
    feature_cols.remove(name)

# Итоговый список признаков
feature_cols

['date',
 'time',
 'temp_pred',
 'year',
 'month',
 'day_of_week',
 'day',
 'holidays_true',
 'preholidays_true',
 'temp_last_day',
 'target_lag_24',
 'target_lag_72',
 'target_lag_336',
 'VVP',
 'cloudy',
 'rainy',
 'windy',
 'clear',
 'rain_probability',
 'has_rain_probability',
 'P',
 'U',
 'WW',
 'Td',
 'N',
 'S',
 'W',
 'E',
 'last_evening_avg_target_0',
 'last_evening_avg_temp_0',
 'last_evening_avg_target_19',
 'last_evening_avg_temp_19',
 'last_evening_avg_target_22',
 'last_evening_avg_temp_22',
 'time_1',
 'temp_pred_1',
 'year_1',
 'month_1',
 'day_of_week_1',
 'day_1',
 'holidays_true_1',
 'preholidays_true_1',
 'temp_last_day_1',
 'target_lag_24_1',
 'target_lag_72_1',
 'target_lag_336_1',
 'VVP_1',
 'cloudy_1',
 'rainy_1',
 'windy_1',
 'clear_1',
 'rain_probability_1',
 'has_rain_probability_1',
 'P_1',
 'U_1',
 'WW_1',
 'Td_1',
 'N_1',
 'S_1',
 'W_1',
 'E_1',
 'last_evening_avg_target_0_1',
 'last_evening_avg_temp_0_1',
 'last_evening_avg_target_19_1',
 'last_evening_avg

In [ ]:
# Итоговый набор колонок
train_ds.columns

#### 1.11.13 Аугументации

In [22]:

feature_aug = [
'temp_pred',
'temp_last_day',
'target_lag_24',
'target_lag_72',
'target_lag_336',
'VVP',
'P',
'U',
'Td',
'last_evening_avg_target_0',
'last_evening_avg_temp_0',
'last_evening_avg_target_19',
'last_evening_avg_temp_19',
'last_evening_avg_target_22',
'last_evening_avg_temp_22',
'temp_pred_1',
'temp_last_day_1',
'target_lag_24_1',
'target_lag_72_1',
'target_lag_336_1',
'VVP_1',
'P_1',
'U_1',
'Td_1',
'last_evening_avg_target_0_1',
'last_evening_avg_temp_0_1',
'last_evening_avg_target_19_1',
'last_evening_avg_temp_19_1',
'last_evening_avg_target_22_1',
'last_evening_avg_temp_22_1',
'temp_pred_2',
'temp_last_day_2',
'target_lag_24_2',
'target_lag_72_2',
'target_lag_336_2',
'VVP_2',
'P_2',
'U_2',
'Td_2',
'last_evening_avg_target_0_2',
'last_evening_avg_temp_0_2',
'last_evening_avg_target_19_2',
'last_evening_avg_temp_19_2',
'last_evening_avg_target_22_2',
'last_evening_avg_temp_22_2',
'temp_pred_3',
'temp_last_day_3',
'target_lag_24_3',
'target_lag_72_3',
'target_lag_336_3',
'VVP_3',
'P_3',
'U_3',
'Td_3',
'last_evening_avg_target_0_3',
'last_evening_avg_temp_0_3',
'last_evening_avg_target_19_3',
'last_evening_avg_temp_19_3',
'last_evening_avg_target_22_3',
'last_evening_avg_temp_22_3',
'temp_pred_4',
'temp_last_day_4',
'target_lag_24_4',
'target_lag_72_4',
'target_lag_336_4',
'VVP_4',
'P_4',
'U_4',
'Td_4',
'last_evening_avg_target_0_4',
'last_evening_avg_temp_0_4',
'last_evening_avg_target_19_4',
'last_evening_avg_temp_19_4',
'last_evening_avg_target_22_4',
'last_evening_avg_temp_22_4',
'temp_pred_5',
'temp_last_day_5',
'target_lag_24_5',
'target_lag_72_5',
'target_lag_336_5',
'VVP_5',
'P_5',
'U_5',
'Td_5',
'last_evening_avg_target_0_5',
'last_evening_avg_temp_0_5',
'last_evening_avg_target_19_5',
'last_evening_avg_temp_19_5',
'last_evening_avg_target_22_5',
'last_evening_avg_temp_22_5',
'temp_pred_6',
'temp_last_day_6',
'target_lag_24_6',
'target_lag_72_6',
'target_lag_336_6',
'VVP_6',
'P_6',
'U_6',
'Td_6',
'last_evening_avg_target_0_6',
'last_evening_avg_temp_0_6',
'last_evening_avg_target_19_6',
'last_evening_avg_temp_19_6',
'last_evening_avg_target_22_6',
'last_evening_avg_temp_22_6',
'temp_pred_7',
'temp_last_day_7',
'target_lag_24_7',
'target_lag_72_7',
'target_lag_336_7',
'VVP_7',
'P_7',
'U_7',
'Td_7',
'last_evening_avg_target_0_7',
'last_evening_avg_temp_0_7',
'last_evening_avg_target_19_7',
'last_evening_avg_temp_19_7',
'last_evening_avg_target_22_7',
'last_evening_avg_temp_22_7',
'temp_pred_8',
'temp_last_day_8',
'target_lag_24_8',
'target_lag_72_8',
'target_lag_336_8',
'VVP_8',
'P_8',
'U_8',
'Td_8',
'last_evening_avg_target_0_8',
'last_evening_avg_temp_0_8',
'last_evening_avg_target_19_8',
'last_evening_avg_temp_19_8',
'last_evening_avg_target_22_8',
'last_evening_avg_temp_22_8',
'temp_pred_9',
'temp_last_day_9',
'target_lag_24_9',
'target_lag_72_9',
'target_lag_336_9',
'VVP_9',
'P_9',
'U_9',
'Td_9',
'last_evening_avg_target_0_9',
'last_evening_avg_temp_0_9',
'last_evening_avg_target_19_9',
'last_evening_avg_temp_19_9',
'last_evening_avg_target_22_9',
'last_evening_avg_temp_22_9',
'temp_pred_10',
'temp_last_day_10',
'target_lag_24_10',
'target_lag_72_10',
'target_lag_336_10',
'VVP_10',
'P_10',
'U_10',
'Td_10',
'last_evening_avg_target_0_10',
'last_evening_avg_temp_0_10',
'last_evening_avg_target_19_10',
'last_evening_avg_temp_19_10',
'last_evening_avg_target_22_10',
'last_evening_avg_temp_22_10',
'temp_pred_11',
'temp_last_day_11',
'target_lag_24_11',
'target_lag_72_11',
'target_lag_336_11',
'VVP_11',
'P_11',
'U_11',
'Td_11',
'last_evening_avg_target_0_11',
'last_evening_avg_temp_0_11',
'last_evening_avg_target_19_11',
'last_evening_avg_temp_19_11',
'last_evening_avg_target_22_11',
'last_evening_avg_temp_22_11',
'temp_pred_12',
'temp_last_day_12',
'target_lag_24_12',
'target_lag_72_12',
'target_lag_336_12',
'VVP_12',
'P_12',
'U_12',
'Td_12',
'last_evening_avg_target_0_12',
'last_evening_avg_temp_0_12',
'last_evening_avg_target_19_12',
'last_evening_avg_temp_19_12',
'last_evening_avg_target_22_12',
'last_evening_avg_temp_22_12',
'temp_pred_13',
'temp_last_day_13',
'target_lag_24_13',
'target_lag_72_13',
'target_lag_336_13',
'VVP_13',
'P_13',
'U_13',
'Td_13',
'last_evening_avg_target_0_13',
'last_evening_avg_temp_0_13',
'last_evening_avg_target_19_13',
'last_evening_avg_temp_19_13',
'last_evening_avg_target_22_13',
'last_evening_avg_temp_22_13',
'temp_pred_14',
'temp_last_day_14',
'target_lag_24_14',
'target_lag_72_14',
'target_lag_336_14',
'VVP_14',
'P_14',
'U_14',
'Td_14',
'last_evening_avg_target_0_14',
'last_evening_avg_temp_0_14',
'last_evening_avg_target_19_14',
'last_evening_avg_temp_19_14',
'last_evening_avg_target_22_14',
'last_evening_avg_temp_22_14',
'temp_pred_15',
'temp_last_day_15',
'target_lag_24_15',
'target_lag_72_15',
'target_lag_336_15',
'VVP_15',
'P_15',
'U_15',
'Td_15',
'last_evening_avg_target_0_15',
'last_evening_avg_temp_0_15',
'last_evening_avg_target_19_15',
'last_evening_avg_temp_19_15',
'last_evening_avg_target_22_15',
'last_evening_avg_temp_22_15',
'temp_pred_16',
'temp_last_day_16',
'target_lag_24_16',
'target_lag_72_16',
'target_lag_336_16',
'VVP_16',
'P_16',
'U_16',
'Td_16',
'last_evening_avg_target_0_16',
'last_evening_avg_temp_0_16',
'last_evening_avg_target_19_16',
'last_evening_avg_temp_19_16',
'last_evening_avg_target_22_16',
'last_evening_avg_temp_22_16',
'temp_pred_17',
'temp_last_day_17',
'target_lag_24_17',
'target_lag_72_17',
'target_lag_336_17',
'VVP_17',
'P_17',
'U_17',
'Td_17',
'last_evening_avg_target_0_17',
'last_evening_avg_temp_0_17',
'last_evening_avg_target_19_17',
'last_evening_avg_temp_19_17',
'last_evening_avg_target_22_17',
'last_evening_avg_temp_22_17',
'temp_pred_18',
'temp_last_day_18',
'target_lag_24_18',
'target_lag_72_18',
'target_lag_336_18',
'VVP_18',
'P_18',
'U_18',
'Td_18',
'last_evening_avg_target_0_18',
'last_evening_avg_temp_0_18',
'last_evening_avg_target_19_18',
'last_evening_avg_temp_19_18',
'last_evening_avg_target_22_18',
'last_evening_avg_temp_22_18',
'temp_pred_19',
'temp_last_day_19',
'target_lag_24_19',
'target_lag_72_19',
'target_lag_336_19',
'VVP_19',
'P_19',
'U_19',
'Td_19',
'last_evening_avg_target_0_19',
'last_evening_avg_temp_0_19',
'last_evening_avg_target_19_19',
'last_evening_avg_temp_19_19',
'last_evening_avg_target_22_19',
'last_evening_avg_temp_22_19',
'temp_pred_20',
'temp_last_day_20',
'target_lag_24_20',
'target_lag_72_20',
'target_lag_336_20',
'VVP_20',
'P_20',
'U_20',
'Td_20',
'last_evening_avg_target_0_20',
'last_evening_avg_temp_0_20',
'last_evening_avg_target_19_20',
'last_evening_avg_temp_19_20',
'last_evening_avg_target_22_20',
'last_evening_avg_temp_22_20',
'temp_pred_21',
'temp_last_day_21',
'target_lag_24_21',
'target_lag_72_21',
'target_lag_336_21',
'VVP_21',
'P_21',
'U_21',
'Td_21',
'last_evening_avg_target_0_21',
'last_evening_avg_temp_0_21',
'last_evening_avg_target_19_21',
'last_evening_avg_temp_19_21',
'last_evening_avg_target_22_21',
'last_evening_avg_temp_22_21',
'temp_pred_22',
'temp_last_day_22',
'target_lag_24_22',
'target_lag_72_22',
'target_lag_336_22',
'VVP_22',
'P_22',
'U_22',
'Td_22',
'last_evening_avg_target_0_22',
'last_evening_avg_temp_0_22',
'last_evening_avg_target_19_22',
'last_evening_avg_temp_19_22',
'last_evening_avg_target_22_22',
'last_evening_avg_temp_22_22',
'temp_pred_23',
'temp_last_day_23',
'target_lag_24_23',
'target_lag_72_23',
'target_lag_336_23',
'VVP_23',
'P_23',
'U_23',
'Td_23',
'last_evening_avg_target_0_23',
'last_evening_avg_temp_0_23',
'last_evening_avg_target_19_23',
'last_evening_avg_temp_19_23',
'last_evening_avg_target_22_23',
'last_evening_avg_temp_22_23',
'target_1',
'temp_1',
'target_5',
'temp_5',
'target_9',
'temp_9']

In [23]:
feature_target = feature_cols +['target']
#feature_target.remove('date')
print(feature_target)

train_ds[feature_aug].dtypes

['date', 'time', 'temp_pred', 'year', 'month', 'day_of_week', 'day', 'holidays_true', 'preholidays_true', 'temp_last_day', 'target_lag_24', 'target_lag_72', 'target_lag_336', 'VVP', 'cloudy', 'rainy', 'windy', 'clear', 'rain_probability', 'has_rain_probability', 'P', 'U', 'WW', 'Td', 'N', 'S', 'W', 'E', 'last_evening_avg_target_0', 'last_evening_avg_temp_0', 'last_evening_avg_target_19', 'last_evening_avg_temp_19', 'last_evening_avg_target_22', 'last_evening_avg_temp_22', 'time_1', 'temp_pred_1', 'year_1', 'month_1', 'day_of_week_1', 'day_1', 'holidays_true_1', 'preholidays_true_1', 'temp_last_day_1', 'target_lag_24_1', 'target_lag_72_1', 'target_lag_336_1', 'VVP_1', 'cloudy_1', 'rainy_1', 'windy_1', 'clear_1', 'rain_probability_1', 'has_rain_probability_1', 'P_1', 'U_1', 'WW_1', 'Td_1', 'N_1', 'S_1', 'W_1', 'E_1', 'last_evening_avg_target_0_1', 'last_evening_avg_temp_0_1', 'last_evening_avg_target_19_1', 'last_evening_avg_temp_19_1', 'last_evening_avg_target_22_1', 'last_evening_avg_t

temp_pred         float64
temp_last_day     float64
target_lag_24     float64
target_lag_72     float64
target_lag_336    float64
                   ...   
temp_1            float64
target_5          float64
temp_5            float64
target_9          float64
temp_9            float64
Length: 366, dtype: object

In [24]:
def augment_row(df_to_augment, features_to_augment, alpha, i, random_1):


    random.seed(random_1*i)

    random_sample1 = random.randint(0000, 50000)
    random_sample2 = random.randint(50001, 100000)

    df_sample1 = df_to_augment.sample(frac=1,
                                     random_state=random_sample1
                                     )
    df_sample2 = df_to_augment.sample(frac=1,
                                     random_state = random_sample2
                                     )

    lmbda = np.random.beta(alpha, alpha)

    df_mixup_sample = df_sample1.copy()
    df_mixup_sample[features_to_augment] = df_sample1[features_to_augment] * lmbda + df_sample2[features_to_augment] * (1 - lmbda)
        
    other_features = list(set(df_to_augment.columns) - set(features_to_augment))
    df_mixup_sample[other_features] = df_sample1[other_features]

    return df_mixup_sample

def mixup(df, alpha, features_to_augment, n_augmentations):
    random.seed(random_state) #random.seed(42)
    random_1 = random.randint(500, 9500)

    
    df_to_augment = df[(df['date'] < open_test_begin)]
    df_to_keep = df[(df['date'] >= open_test_begin)]

    df_mixup = pd.concat([augment_row(df_to_augment, features_to_augment, alpha, i, random_1) for i in range(n_augmentations)]).parallel_apply(lambda x: x)

    df_final = pd.concat([ df_mixup,
                           df_to_augment,  #нужно раскомитить если обучение с нуля
                           df_to_keep])

    return df_final


n_augmentations = 27
alpha=0.3 #15
n_frac = 1
train_ds_mixup = mixup(train_ds[feature_target], alpha, feature_aug, n_augmentations)
FEATURES = '_Aug_27_alpha_3'
num = 6 #номер модели

In [31]:
print(train_ds[feature_target].shape, train_ds_mixup.shape)
train_ds_mixup.reset_index(inplace=True)

(40027, 800) (1041943, 801)


/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_37729/132879785.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_ds_mixup.reset_index(inplace=True)


#### 1.12 Выделение наборов данных для обучения, валидации и тестирования

Выделялось два набора данных для обучения и валидации:
1. Обучение на данных с 2019 по 2021 с валидацией на 2022
2. Обучение на данных с 2019 по 2022 с валидацией на первом квартале 2023

Первый набор позволяет оценить влияние сезонности на обучение и предсказания, второй позволяет обучить модель на большем объеме данных и на более актуальных данных.

In [26]:
# Формируем набор датасетов для обучения и проверки
features = train_ds_mixup[feature_cols]
target = train_ds_mixup['target']

# Функция для выделения временных интервалов из таблиц признаков и целей
# на этом этапе отбрасываем колонку 'date'
def features_interval(features, target, date1, date2):
    features_interval = features[ (features['date']>=date1) & (features['date']<date2) ]
    target_interval = target[features_interval.index]
    features_interval = features_interval.drop('date', axis=1)
    return features_interval, target_interval



# для первичного подбора гиперпараметров будем обучать на 19-22 годах, валидировать август-сентябрь 2022
features_all_train, target_all_train = features_interval(features, target, '2019-01-01', '2022-08-01')
features_open_test, target_open_test = features_interval(features, target, '2022-08-01', '2022-09-30')

# для проверки на тестовой выборке будем учиться на всем тренировочном датасете
features_all_train, target_all_train = features_interval(features, target, '2019-01-01', open_test_begin)
features_open_test, target_open_test = features_interval(features, target, open_test_begin, open_test_end)

features_open_test_sum, target_open_test_sum = features_interval(features, target, '2023-06-01', open_test_end)





### 4. Проверка метрик на тестовом датасете

#### 4.1 LGBM

In [27]:
features_all_train.columns

Index(['time', 'temp_pred', 'year', 'month', 'day_of_week', 'day',
       'holidays_true', 'preholidays_true', 'temp_last_day', 'target_lag_24',
       ...
       'last_evening_avg_target_19_23', 'last_evening_avg_temp_19_23',
       'last_evening_avg_target_22_23', 'last_evening_avg_temp_22_23',
       'target_1', 'temp_1', 'target_5', 'temp_5', 'target_9', 'temp_9'],
      dtype='object', length=798)

In [28]:
drop_list = [#'preholidays',
            #'has_rain_probability', 'W', 'E',
            #'holidays' 
            ]
feat_lgbm_train = features_all_train.drop(columns=drop_list)
feat_lgbm_test = features_open_test.drop(columns=drop_list)
print(feat_lgbm_train.shape, target_all_train.shape, feat_lgbm_test.shape)

(1041943, 798) (1041943,) (2919, 798)


In [32]:
# Проверка метрики лучшей модели на тестовом датасете
# Здесь обучаем на всем тренировочном датасете



params = {'num_leaves':15, 
          'learning_rate':0.02, 
          'feature_fraction':1, 
          'num_iterations':10000, 
          'random_state':random_state, 
          'objective':'regression_l1',
          'n_jobs':-1}
params= {'num_leaves': 34, 'min_child_samples': 16, 
          'max_depth': 8, 'learning_rate': 0.012, 
          'min_sum_hessian_in_leaf': 1e-4,
          'objective': 'regression_l1', 'feature_fraction': 0.9574152630927155,
          'n_jobs':-1, 'num_iterations':10000
          }


def print_iteration_info(env):
    # выводим номер итерации и время, затраченное на предыдущую итерацию
    print('Iteration:', env.iteration)


lgbm_model_all_train = lgb.LGBMRegressor(**params)
lgbm_model_all_train.fit(feat_lgbm_train, target_all_train, callbacks=[print_iteration_info])

l_predict_train = lgbm_model_all_train.predict(feat_lgbm_train)
l_predict_test = lgbm_model_all_train.predict(feat_lgbm_test)

mae_train, mape_train, r2_train = metrics_hour(target_all_train, l_predict_train)
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, l_predict_test)

results = pd.DataFrame([[f'тренировочная LGBM {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая LGBM {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))

/opt/anaconda3/envs/p311/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


KeyboardInterrupt: 

# До обучение модели 


params= {'num_leaves': 34, 'min_child_samples': 16, 
          'max_depth': 8, 'learning_rate': 0.012, 
          'min_sum_hessian_in_leaf': 1e-4,
          'objective': 'regression_l1', 'feature_fraction': 0.9574152630927155,
          'n_jobs':-1, 'num_iterations':10000
          }

def print_iteration_info(env):
    # выводим номер итерации и время, затраченное на предыдущую итерацию
    print('Iteration:', env.iteration)



lgbm_model_all_train.fit(feat_lgbm_train, target_all_train, callbacks=[print_iteration_info],  init_model=lgbm_model_all_train)

l_predict_train = lgbm_model_all_train.predict(feat_lgbm_train)
l_predict_test = lgbm_model_all_train.predict(feat_lgbm_test)

mae_train, mape_train, r2_train = metrics_hour(target_all_train, l_predict_train)
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, l_predict_test)

results = pd.DataFrame([[f'тренировочная LGBM add train {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая LGBM add train {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))

In [30]:
results

,Выборка,MAE,MAPE,R2
0,тренировочная LGBM _Aug_27_alpha_3,3.335556,0.006983,0.996601
1,тестовая LGBM _Aug_27_alpha_3,5.652768,0.013195,0.987176


#### 4.2 XGBoost

In [ ]:
features_all_train.columns

In [ ]:
drop_list = ['preholidays_true',
            #'has_rain_probability', 
            # 'W', 'E'
            ]
n_values = range(1, 24)
preholidays = ['preholidays_true_{}'.format(n) for n in n_values]
#has_rain = ['has_rain_probability_{}'.format(n) for n in n_values]
#W_wind = ['W_{}'.format(n) for n in n_values]
#E_wind = ['E_{}'.format(n) for n in n_values]

drop_list = drop_list + preholidays #+ has_rain + W_wind + E_wind

feat_xgb_train = features_all_train.drop(columns=drop_list)
feat_xgb_test = features_open_test.drop(columns=drop_list)
feat_xgb_train.columns, feat_xgb_test.columns

In [ ]:
xgb_model = XGBRegressor(
    max_depth=7,
    n_estimators=1190, #n_estimators=195, #
    learning_rate=0.009, #learning_rate=0.1, #
    tree_method='exact',
    objective='reg:squarederror',
    eval_metric='rmse',
    gamma=2,
    colsample_bytree=1,
    random_state=random_state

)


In [ ]:
# Проверка метрики лучшей модели на тестовом датасете


class IterationInfoCallback(TrainingCallback):
    def __init__(self):
        self.start_time = time.time()

    def after_iteration(self, model, epoch, evals_log):
        print('Iteration:', epoch, 'Time for last iteration:', self.start_time - time.time())
        self.start_time = time.time()
        return False



xgb_model_all_train = xgb_model.fit(feat_xgb_train, target_all_train, callbacks=[IterationInfoCallback()])

xgb_predict_test = xgb_model_all_train.predict(feat_xgb_test)
xgb_predict_train = xgb_model_all_train.predict(feat_xgb_train)

mae_train, mape_train, r2_train = metrics_hour(target_all_train, xgb_predict_train )
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, xgb_predict_test )

results = pd.concat([results,
pd.DataFrame([[f'тренировочная XGB {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая XGB {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])


In [ ]:
drop_list = ['preholidays_true',
            ]
n_values = range(1, 24)
preholidays = ['preholidays_true_{}'.format(n) for n in n_values]


drop_list = drop_list + preholidays #+ has_rain + W_wind + E_wind

feat_xgb_test_summ = features_open_test_sum.drop(columns=drop_list)


In [ ]:

xgb_predict_test_sum = xgb_model_all_train.predict(feat_xgb_test_summ)
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test_sum, xgb_predict_test_sum )
results = pd.concat([results,
pd.DataFrame([ [f'тестовая XGB summer{FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])

In [ ]:
results

### 5. Ensemble

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNet
from lightgbm import LGBMRegressor
from sklearn.ensemble import StackingRegressor

import optuna
from optuna.samplers import TPESampler

In [ ]:
results_ensemble = pd.DataFrame(columns=('Выборка', 'MAE', 'MAPE', 'R2'))

### Simple Ensemble

In [ ]:
predict_simple_ensemble_train = (xgb_predict_train + l_predict_train)/2
predict_simple_ensemble_test = (xgb_predict_test + l_predict_test)/2



In [ ]:
mae_train, mape_train, r2_train = metrics_hour(target_all_train, predict_simple_ensemble_train)
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, predict_simple_ensemble_test)

results_ensemble = pd.concat([results_ensemble,
pd.DataFrame([[f'тренировочная simple_ensemble  {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая simple_ensemble {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])

In [ ]:
display(results_ensemble)
results

In [ ]:
mae_day(target_open_test, predict_simple_ensemble_test)

In [ ]:
import os
#num = 3
# определите путь к папке, которую вы хотите создать
folder_path = "models"

# проверьте, существует ли уже папка
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

lgbm_model_all_train.booster_.save_model(f'models/lgb_model{FEATURES}_{num}.txt')
xgb_model.save_model(f'models/xgb_model{FEATURES}_{num}.json')

In [ ]:
#FEATURES = 'BASE'
results.to_csv(f'models/results_LGBM_XGBoost_simple_ensemble_{FEATURES}_{num}')